<a href="https://colab.research.google.com/github/Dumh1233/MathWise/blob/test_segmentation/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import tensorflow as tf
import os
import numpy as np
from PIL import Image
import xml.etree.ElementTree as ET
!pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
!gdown --id 1-GuEQIk4irl4wiDT8NYAsJgjAnIGqKFA
!unzip mathwise_dataset.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-GuEQIk4irl4wiDT8NYAsJgjAnIGqKFA
From (redirected): https://drive.google.com/uc?id=1-GuEQIk4irl4wiDT8NYAsJgjAnIGqKFA&confirm=t&uuid=ea014fb6-22c3-4882-8e1a-cbfc1f12b14a
To: /content/mathwise_dataset.zip
100% 772M/772M [00:09<00:00, 83.0MB/s]
Archive:  mathwise_dataset.zip
   creating: Dataset/
   creating: Dataset/ground truth/
  inflating: Dataset/ground truth/10.1.1.1.2004_4.xml  
  inflating: Dataset/ground truth/10.1.1.1.2004_5.xml  
  inflating: Dataset/ground truth/10.1.1.1.2005_12.xml  
  inflating: Dataset/ground truth/10.1.1.1.2005_13.xml  
  inflating: Dataset/ground truth/10.1.1.1.2005_4.xml  
  inflating: Dataset/ground truth/10.1.1.1.2005_7.xml  
  inflating: Dataset/ground truth/10.1.1.1.2007_11.x

In [16]:
# Define the paths to the preprocessed images and annotations
images_path = "Dataset/image"
annotations_path = "Dataset/ground truth"

In [17]:
# Define the output directory for the trained model
model_output_path = "output"

In [18]:
# Define the input shape for the model
input_shape = (512, 512)

In [19]:
# Define the batch size and number of epochs for training
batch_size = 8
num_epochs = 50

In [20]:
# Define the number of classes (equation or non-equation)
num_classes = 2

In [21]:
# Define the learning rate and optimizer for training
learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [22]:
# Define the model architecture
inputs = tf.keras.layers.Input(shape=input_shape + (3,))
base_model = tf.keras.applications.ResNet50(include_top=False, input_tensor=inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
model = tf.keras.models.Model(inputs, outputs)

In [23]:
# Compile the model with binary cross-entropy loss and accuracy metric
model.compile(
    loss="binary_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"]
)

In [24]:
# Define the data generator for the training and validation sets
def data_generator(images_path, annotations_path, batch_size, input_shape):
    image_filenames = os.listdir(images_path)
    annotation_filenames = os.listdir(annotations_path)
    while True:
        X = np.zeros((batch_size,) + input_shape + (3,))
        Y = np.zeros((batch_size, 1))
        for i in range(batch_size):
            # Choose a random image and annotation file
            index = np.random.randint(len(image_filenames))
            image_filename = image_filenames[index]
            annotation_filename = annotation_filenames[index]
            # Load the image and resize it to the input shape
            image_path = os.path.join(images_path, image_filename)
            image = Image.open(image_path)
            image = image.resize(input_shape)
            image_array = np.array(image) / 255.0
            X[i] = image_array
            # Load the annotation file and extract the bounding box and label of the equation
            annotation_path = os.path.join(annotations_path, annotation_filename)
            tree = ET.parse(annotation_path)
            root = tree.getroot()
            for equation in root.iter("equation"):
                x = float(equation.get("x"))
                y = float(equation.get("y"))
                width = float(equation.get("width"))
                height = float(equation.get("height"))
                label = 1  # equation
                y_min = int(y * input_shape[0])
                x_min = int(x * input_shape[1])
                y_max = int((y + height) * input_shape[0])
                x_max = int((x + width) * input_shape[1])
                Y[i] = np.array([y_min, x_min, y_max, x_max, label])
        yield X, Y

In [25]:
# Split the dataset into training and validation sets
train_generator = data_generator(images_path, annotations_path, batch_size, input_shape)
validation_generator = data_generator(images_path, annotations_path, batch_size, input_shape)

In [26]:
history = model.fit(train_generator, validation_data=validation_generator, batch_size=batch_size, epochs=num_epochs)

Epoch 1/50
    523/Unknown - 3224s 6s/step - loss: 0.0022 - accuracy: 0.4536

KeyboardInterrupt: ignored